<a href="https://colab.research.google.com/github/AlinaSabitova/-_-/blob/main/analyze_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
Анализ зарплат в Data Science с использованием Pandas
Задача: найти среднюю зарплату по уровню опыта
"""
import pandas as pd
import sys
import os

def load_data(filepath):
    """Загрузить данные из CSV файла"""
    try:
        df = pd.read_csv(filepath, low_memory=False)
        print(f"Загружено строк: {len(df)}")
        return df
    except Exception as e:
        print(f"Ошибка при загрузке данных: {e}")
        sys.exit(1)

def clean_data(df):
    """Очистка и подготовка данных"""
    print("\n=== Очистка данных ===")
    print(f"Исходное количество строк: {len(df)}")

    # Удалить строки без зарплаты
    df = df[df['salary_in_usd'].notna()]

    # Заполнить пустые значения в experience_level
    df['experience_level'] = df['experience_level'].fillna('Unknown')

    # Расшифровка кодов уровней опыта
    experience_map = {
        'EN': 'Entry-level',
        'MI': 'Mid-level',
        'SE': 'Senior-level',
        'EX': 'Executive-level',
        'Unknown': 'Unknown'
    }
    df['experience_level_name'] = df['experience_level'].map(experience_map)

    print(f"Количество строк после очистки: {len(df)}")
    print(f"Уникальных уровней опыта: {df['experience_level'].nunique()}")

    return df

def analyze_salary_by_experience(df):
    """Анализ средней зарплаты по уровням опыта"""
    print("\n=== Анализ средней зарплаты по уровням опыта ===")

    # Группировка по уровню опыта и вычисление статистики по зарплате
    result = df.groupby(['experience_level', 'experience_level_name'])['salary_in_usd'].agg([
        'mean', 'count', 'std', 'min', 'max', 'median'
    ]).reset_index()

    result.columns = [
        'Experience_Code', 'Experience_Level', 'Mean_Salary_USD',
        'Count', 'Std_Deviation', 'Min_Salary', 'Max_Salary', 'Median_Salary'
    ]

    # Сортировка по средней зарплате
    result = result.sort_values('Mean_Salary_USD', ascending=False)

    return result

def find_salary_statistics(df):
    """Найти статистику зарплат по уровням опыта"""
    result = analyze_salary_by_experience(df)

    print("\n=== Результаты анализа зарплат ===")
    print("\nУровни опыта по средней зарплате:")
    print(result.to_string(index=False, float_format='%.2f'))

    max_salary_level = result.iloc[0]
    min_salary_level = result.iloc[-1]

    print(f"\nСамая высокооплачиваемая категория: '{max_salary_level['Experience_Level']}' ({max_salary_level['Experience_Code']})")
    print(f"Средний доход по категории: ${max_salary_level['Mean_Salary_USD']:,.2f} USD")
    print(f"Количество специалистов данной категории: {int(max_salary_level['Count'])}")
    print(f"Диапазон зарплат в данной категории: ${max_salary_level['Min_Salary']:,.2f} - ${max_salary_level['Max_Salary']:,.2f} USD")

    print(f"\Наименее оплачиваемая категория: '{min_salary_level['Experience_Level']}' ({min_salary_level['Experience_Code']})")
    print(f"Средний доход по категории: ${min_salary_level['Mean_Salary_USD']:,.2f} USD")
    print(f"Количество специалистов данной категории: {int(min_salary_level['Count'])}")
    print(f"Диапазон зарплат в данной категории: ${min_salary_level['Min_Salary']:,.2f} - ${min_salary_level['Max_Salary']:,.2f} USD")

    return result

def additional_analysis(df):
    """Дополнительный анализ данных"""
    print("\n=== Дополнительная статистика ===")

    # Общая статистика по зарплатам
    print(f"\nОбщая статистика зарплат:")
    print(f"Средняя зарплата: ${df['salary_in_usd'].mean():,.2f}")
    print(f"Минимальная зарплата: ${df['salary_in_usd'].min():,.2f}")
    print(f"Максимальная зарплата: ${df['salary_in_usd'].max():,.2f}")

    # Распределение по уровням опыта
    print(f"\nРаспределение по уровням опыта:")
    experience_counts = df['experience_level_name'].value_counts()
    for level, count in experience_counts.items():
        percentage = (count / len(df)) * 100
        print(f"  {level}: {count} специалистов ({percentage:.1f}%)")

def main():
    # Путь к файлу данных
    data_file = '/opt/data/salary_data.csv'

    if not os.path.exists(data_file):
        # Попробовать локальный путь
        data_file = 'hdfs://localhost:9000/user/hadoop/input/salary_data.csv'

    if not os.path.exists(data_file):
        print(f"Файл не найден: {data_file}")
        print("Убедитесь, что файл загружен в HDFS и доступен по правильному пути")
        sys.exit(1)

    print("=== Анализ зарплат в Data Science ===")
    print(f"Файл: {data_file}")

    # Загрузка данных
    df = load_data(data_file)

    # Показать базовую информацию
    print("\n=== Информация о данных ===")
    print(f"Колонки: {list(df.columns)}")
    print(f"Общее количество записей: {len(df)}")

    print("\nПервые 5 строк:")
    print(df.head())

    # Очистка данных
    df_clean = clean_data(df)

    # Основной анализ
    result = find_salary_statistics(df_clean)

    # Дополнительный анализ
    additional_analysis(df_clean)

    # Сохранить результаты
    output_file = 'results/salary_by_experience.csv'
    os.makedirs('results', exist_ok=True)
    result.to_csv(output_file, index=False)
    print(f"\nРезультаты сохранены в: {output_file}")

    return result

if __name__ == '__main__':
    main()